In [27]:
%load_ext autoreload
%autoreload 2
import refactorings
from pathlib import Path
import difflib
import random
import pandas as pd
import random
import tqdm.notebook as tqdm
import tempfile
from contextlib import redirect_stdout
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
df = pd.read_json('/home/benja/work/weile-lab/thesis/ReVeal/data/ggnn_input/devign_cfg_full_text_files.json')
i, functions = zip(*list(df.iterrows()))
print(len(functions), 'samples')

# functions = functions[:24]
# print('cutting to', len(functions), 'samples')

func_lines = functions[0]["code"].splitlines()
print('Example:')
print('\n'.join(func_lines[:3]))
print('...')
print('\n'.join(func_lines[-3:]))

26733 samples
Example:
static int tight_compress_data(VncState *vs, int stream_id, size_t bytes,
                                int level, int strategy)
 {
...
     buffer_reset(&vs->tight.zlib);
     return bytes;
 }


In [50]:
def do_one(fn):
    print(fn["file_name"])
    code = fn["code"]
    with open(tmp_file, 'w') as f:
        f.write(code)
    with factory.make_project(tmp_file) as project:
        new_file, applied = project.apply_all(return_applied=True)
        with open(new_file) as f:
            new_fn = f.read()
            # new_functions.append((new_fn, applied))
            return (new_fn, applied)
    # if len(new_functions) > 5000:
    #     with open(f'new_functions.pkl.shard{shards}', 'wb') as f:
    #         pickle.dump(new_functions, f)
    #     shards += 1
    #     new_functions = []

import multiprocessing
import itertools
factory = refactorings.TransformationsFactory(refactorings.all_refactorings, refactorings.random_picker)
with tempfile.TemporaryDirectory() as tmp_dir:
    shards = 0
    # new_functions = []
    shard_len = 5000

    tmp_dir = Path(tmp_dir)
    tmp_file = tmp_dir / 'tmp.c'
    print('Temporary file:', tmp_file)
    
    log_filename = 'log.txt'
    print('Redirecting to', log_filename)
    with open(log_filename, 'w') as f:
        with redirect_stdout(f):
            with multiprocessing.Pool(8) as p:
                it = tqdm.tqdm(p.imap(do_one, functions), total=len(functions))
                while True:
                    new_functions = list(itertools.islice(it, shard_len))
                    if len(new_functions) == 0:
                        break
                    else:
                        shard_filename = f'new_functions.pkl.shard{shards}'
                        print('Saving shard', shard_filename, 'size', len(new_functions))
                        with open(shard_filename, 'wb') as f:
                            pickle.dump(new_functions, f)
                        shards += 1
                        del new_functions

Temporary file: /tmp/tmp7_dviv0m/tmp.c
Redirecting to log.txt


Process ForkPoolWorker-31:
Process ForkPoolWorker-25:
Process ForkPoolWorker-30:
Process ForkPoolWorker-26:
Process ForkPoolWorker-29:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-27:
  File "/home/benja/miniconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/benja/miniconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-28:
Process ForkPoolWorker-32:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/benja/miniconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/benja/miniconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/benja/miniconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwar

KeyboardInterrupt: 

In [34]:
import difflib
for i in range(len(new_functions)):
    print('Applied:', [x.__name__ for x in new_functions[i][1]])
    print(''.join(difflib.unified_diff(functions[i].splitlines(keepends=True), new_functions[i][0].splitlines(keepends=True))))

Applied: ['InsertNoop', 'PermuteStmt', 'RenameVariable']
--- 
+++ 
@@ -1,8 +1,8 @@
 static int tight_compress_data(VncState *vs, int stream_id, size_t bytes,
                                 int level, int strategy)
  {
-     z_streamp zstream = &vs->tight.stream[stream_id];
      int previous_out;
+     z_streamp fungus = &vs->tight.stream[stream_id];
      if (bytes < VNC_TIGHT_MIN_TO_COMPRESS) {
          vnc_write(vs, vs->tight.tight.buffer, vs->tight.tight.offset);
          return bytes;
@@ -13,19 +13,20 @@
      /* reserve memory in output buffer */
      buffer_reserve(&vs->tight.zlib, bytes + 64);
      /* set pointers */
-     zstream->next_in = vs->tight.tight.buffer;
-     zstream->avail_in = vs->tight.tight.offset;
-     zstream->next_out = vs->tight.zlib.buffer + vs->tight.zlib.offset;
-     zstream->avail_out = vs->tight.zlib.capacity - vs->tight.zlib.offset;
-     zstream->data_type = Z_BINARY;
-     previous_out = zstream->total_out;
+     fungus->next_in = vs->tight.t

In [47]:
import itertools
it = iter([1, 2, 3, 4, 5])
print(list(itertools.islice(it, 3)))
print(list(itertools.islice(it, 3)))
print(list(itertools.islice(it, 3)))

[1, 2, 3]
[4, 5]
[]
